# Generating Debug Certificate with a Signature Provider for MIMXRT595S

This notebook outlines the process of generating a debug credentials file for debug authentication using a custom remote signing service. We will detail the steps necessary to create a debug certificate, ensuring secure and efficient authentication.

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)
- Connect the mimxrt595s-evk board via debugger

  <img src="../../_data/img/boards/mimxrt595s-evk.png" alt="mimxrt595s-evk" height="300">

Let's prepare also workspace and variables.

In [1]:
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
%env JUPYTER_SPSDK=1
# Set a magic for command execution and echo
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [2]:
import os

DATA_DIR = "_data/"
WORKSPACE = "workspace/"  # change this to path to your workspace
PLUGINS_DIR = "../_common/plugins/"
FAMILY = "mimxrt595s"
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

## 2. Signature Provider Plugin Setup

First, we need to setup the Signature Provider plugin and start the custom HSM. In order to do that, go to [Signature Provider notebook](../_common/signature_provider.ipynb) and follow the instructions there. 
Once you are done, come back and continue here.

## 3. Preparing the configuration file

Let's begin with creating a template configuration file using the `nxpdebugmbox dat dc get-template` command. This command generates a YAML template that can be customized so the custom Signature Provider is integrated. Below, we'll compare the differences between the template and our customized example to highlight the additions we've made.

In [4]:
# Get difference of template and user YAML configuration
YamlDiffWidget("mimxrt595s_dat.diffc").html

nxpdebugmbox dat dc get-template -f mimxrt595s -o workspace/mimxrt595s_template.yaml --force 
The Debug Credentials template for mimxrt595s, Revision: latest has been saved into workspace/mimxrt595s_template.yaml YAML file


### 3.1 Signature Provider configuration

The signature provider configuration string must follow the format:

  ```
  "type=<identifier>;<key1>=<value1>;<key2>=<value2>;..."
  ```
 - The `<identifier>` must match the `identifier` class attribute defined in the custom Signature Provider (in this example `plugins/spsdk_sasp/spsdk_sasp/provider.py`).
 - The remaining key-value pairs will be passed to the `__init__` method of the given Signature Provider.


For instance, the configuration string:
  ```
  "type=file;file_path=private_key.pem"
  ```  
  will instantiate the following object:  
  ```python
  spsdk.crypto.PlainFileSP(file_path='private_key.pem')
  ```

## 4. Debug Certificate Generation

At this point, we have everything we need to run create debug certificate signed by remote HSM.

In [4]:
CONFIG_PATH = os.path.join(DATA_DIR, "dck_rsa_2048.yaml")
OUTPUT_PATH = os.path.join(WORKSPACE, "dc_out.cert")
%! nxpdebugmbox $VERBOSITY dat dc export --config $CONFIG_PATH --output $OUTPUT_PATH --force

# check if the signed image exists
assert os.path.exists(OUTPUT_PATH)

nxpdebugmbox -v dat dc export --config _data/dck_rsa_2048.yaml --output workspace/dc_out.cert --force 
INFO:spsdk.apps.nxpdebugmbox:Loading configuration from yml file...
INFO:spsdk.apps.nxpdebugmbox:Creating RSA debug credential object...
RKTH: 5905022784a39901b0dc0860c9455cd1b83c5336a2e973825759961554664c89
INFO:spsdk.apps.nxpdebugmbox:Saving the debug credential to a file: workspace/dc_out.cert
Creating Debug credential file succeeded


## 5. HSM teardown

Last step is to stop custom HSM. In order to do that, open again the [HSM Setup notebook](../_common/sahsm.ipynb) and stop the running jupyter notebook code cell.